In [1]:
from functools import wraps
from pathlib import Path
from typing import Union, List, Dict
import os
import json
from multiprocessing import Pool
from pprint import pprint

import numpy as np
import openfoamparser_mai as Ofpp
import pyvista

In [2]:
LOW_DIM_PATH = 'low_dim'
HIGH_DIM_PATH = 'high_dim'
CONSTANT_PATH = 'constant'
POLYMESH_PATH = 'polyMesh'
SYSTEM_PATH = 'system'

In [3]:
def get_vel_data(
    vel: str,
    base_path: str = 'data',
    model: str = 'data_wage',
    dimpath: str = 'low_dim',
) -> Dict[str, List[np.ndarray]]:
    parsepath = Path(
        '/'.join([
            base_path,
            model,
            dimpath,
            vel,
        ])
    )
    time_paths = sorted(list(parsepath.iterdir()))[1:-2]
    veldata = {}
    for time_path in time_paths:
        time_path = Path(time_path)
        p = Ofpp.parse_internal_field(time_path / Path('p'))
        t = Ofpp.parse_internal_field(time_path / Path('T'))
        u = Ofpp.parse_internal_field(time_path / Path('U'))
        rho = Ofpp.parse_internal_field(time_path / Path('rho'))

        key = str(time_path).split('/')[-1]
        veldata[key] = [p, t, u, rho]
    return veldata

In [4]:
def get_all_dim_data(
    base_path: str = 'data',
    model: str = 'data_wage',
    dimpath: str = 'low_dim',
):
    parsepath = Path(
        '/'.join([
            base_path,
            model,
            dimpath,
        ])
    )
    dimdata = {}
    # return sorted(list(parsepath.iterdir()))[1:]
    for velpath in sorted(list(parsepath.iterdir()))[1:]:
        velpath = Path(velpath)
        key = str(velpath).split('/')[-1]
        dimdata[key] = get_vel_data(
            vel=key,
            base_path=base_path,
            model=model,
            dimpath=dimpath,
        )
    mesh: Ofpp.FoamMesh = Ofpp.FoamMesh(parsepath / Path('vel3.0'))

    dimdata['mesh'] = {
        'faces': mesh.faces,
        'boundary': mesh.boundary,
        'neighbour': mesh.neighbour,
        'owner': mesh.owner,
        'points': mesh.points,
    }
    return dimdata

In [5]:
dimdata = get_all_dim_data()

In [8]:
mesh: Ofpp.FoamMesh = dimdata['mesh']

faces = mesh['faces']
np.min(faces)

0

In [9]:
np.max(faces)

191